In [2]:
import numpy as np
import matplotlib.pyplot as plt

import pulp as lp
import itertools
import copy

In [3]:
# System Parameters
T = 300
num_users = 3
num_svrs = 20
space = np.array(([-2.5,2.5],[-2.5,2.5])) # Physical simulation space 2d

# Generate mixtures 
num_modes = 3
modes = np.zeros(num_users)
underlying_dists = np.random.uniform(low=0, high = 1, size = [num_modes, num_svrs])

# User time change points
rng = np.random.default_rng()
min_change_count = 30
max_change_count = 100

usr_dist_changes = {}
for u in range(num_users):
    usr_dist_changes[u] = np.sort(rng.choice(np.arange(1,T-1), 
                                  size=np.random.randint(min_change_count, max_change_count),
                                  replace=False))

# User observation Collection
user_rewards_array = np.zeros([T, num_svrs])
user_pulls_list = np.zeros([T,num_svrs])

# 

In [25]:
# Homogeniety Test Function

def homogeneity_test(num_pulls1, num_pulls2, rewards1, rewards2, sigma_sqd):
    """
    Given two sets of observations (num_pulls, rewards), 
    determine if they are from the same distribution or not
    
    Inputs:
    num_pulls - (kx1) list where k is the number of arms
    rewards - (kx1) list, where each element is a sub-list with reward values from each pull 
    """
    
    k = len(num_pulls1)
    obs1, obs2, obs_both = np.zeros(k), np.zeros(k), np.zeros(k)
    
    for i in range(k):
        total_reward1, total_reward2 = sum(rewards1[i]), sum(rewards2[i])
        obs1[i] = total_reward1/num_pulls1[i]
        obs2[i] = total_reward2/num_pulls2[i]
        obs_both[i] = (total_reward1 + total_reward2)/(num_pulls1[i] + num_pulls2[i])
        
    s = (np.linalg.norm(obs1 - obs_both, ord=2) + np.linalg.norm(obs2 - obs_both, ord=2))/sigma_sqd
    
    return s

In [29]:
num_pulls1 = [2,3,2]
num_pulls2 = [1,2,4]
rewards1 = [[1,2],[3,2,4],[5,3]]
rewards2 = [[3],[7,3],[4,2,6,3]]
sigma_sqd = 1

s = homogeneity_test(num_pulls1, num_pulls2, rewards1, rewards2, sigma_sqd)
s

2.522278409193636

In [31]:
num_pulls1 = [2,3,2]
num_pulls2 = [2,3,2]
rewards1 = [[1,2],[3,2,4],[5,3]]
rewards2 = [[2,2],[4,2,4],[5,2]]
sigma_sqd = 1

s = homogeneity_test(num_pulls1, num_pulls2, rewards1, rewards2, sigma_sqd)
s

0.7817359599705717